### import 및 함수

In [1]:
# dict_kcs = {"kcs" : "pullPath"}
# 20494491752A40E088201 /volume1/솔림헬프/기타/전북-01/20494491/20494491_보증인 오병석_기타_2022본11747_유체접수증.pdf
import os
import pickle
import binascii
import re
from os.path import join
from file_function import read_filelist, re_name_upload, write_log_csv_upload
from tqdm import tqdm

#################################################################
company = "대성"
#################################################################
if company == "솔림" :
    path_kcs = r"\\192.168.0.75\스캔파일\스캔파일log\_project\파일\중복조사\kcs별 파일정보_솔림헬프.pkl"
else  : path_kcs = r"\\192.168.0.75\스캔파일\스캔파일log\_project\대성파일\중복조사\kcs별 파일정보_대성.pkl"

def crc32_checksum(filename):
    buf = open(filename,'rb').read()
    buf = (binascii.crc32(buf) & 0xFFFFFFFF)
    return "%08X" % buf

def get_kcs(path_192) :
    crc32 = str(crc32_checksum(path_192))
    size = str(os.path.getsize(path_192))
    f = os.path.split(path_192)[1]
    key = f[:8]
    kcs = key + crc32 + size 
    return kcs

def read_kcs(path_kcs) :
    with open(path_kcs, "rb") as pkl :
        return pickle.load(pkl)

def delete_kcs(filePath, dict_kcs) :
    k = os.path.split(filePath)[1][:8]
    c = str(crc32_checksum(filePath))
    s = str(os.path.getsize(filePath))
    kcs = k + c + s
    if kcs in dict_kcs :
        print(f"kcs : {kcs}   path : {dict_kcs[kcs]}")
        del dict_kcs[kcs]
    
        

def write_kcs(path_kcs, dict_kcs) :
    with open(path_kcs, "wb") as pkl :
        pickle.dump(dict_kcs, pkl)

# def update_kcs(src, dst, dict_kcs) :
    
#     kcs = get_kcs(src)
#     print(dict_kcs[kcs])

#     new_dst = re.sub(r"\\\\192.168.0.75", "/volume1", dst)
#     new_dst = re.sub(r"\\", "/", new_dst)
#     dict_kcs[kcs] = new_dst
#     print(dict_kcs[kcs], "dict수정 성공")

def update_kcs(src, dst, dict_kcs) :
    # 채무자키
    src_key = os.path.split(src)[1][:8]
    dst_key = os.path.split(dst)[1][:8]
    # kcs
    kcs = get_kcs(src)
    before_value = dict_kcs[kcs] # 수정되기 전에 변수에 담아두어야
    # 윈도우 path를 리눅스 path로
    new_dst = re.sub(r"\\\\192.168.0.75", "/volume1", dst)
    new_dst = re.sub(r"\\", "/", new_dst)

    # 채무자키가 달라졌을 때 > dict의 key를 수정해야 하므로 삭제 후 새값 추가
    if src_key != dst_key :
        new_kcs = dst_key + kcs[8:]
        del dict_kcs[kcs] # 구값 삭제
    else : new_kcs = kcs
    
    # 새값 추가/수정(채무자키 동일했다면 kcs == new_kcs 따라서 수정된다.)
    dict_kcs[new_kcs] = new_dst

    print(f"dict 키 변화 : {kcs} >>> {new_kcs}")
    print(f"dict 값 변화 : {before_value} >>> {dict_kcs[new_kcs]}")
    
print(f'company : {company}맞음?')

company : 대성맞음?


### 단일파일 dict 삭제

In [2]:
# dict 로드
dict_kcs = read_kcs(path_kcs)
print("최초dict_kcs개수 :", len(dict_kcs))

최초dict_kcs개수 : 33405


In [3]:
# dict에서 삭제
####################################
# 에브리띵에서 전체경로복사하면 편함, 채무자키만 유지하면 파일위치, 파일명는 상관없음
path_192 = r"\\192.168.0.75\대성\5.등초본\씨에프-01\10625362\10625362_임혜자_초본.pdf"
####################################
삭제전 = len(dict_kcs)
delete_kcs(path_192, dict_kcs)
삭제후 = len(dict_kcs)
print(f'{삭제전 - 삭제후}건 중복파일 dict_kcs 정보 삭제')

kcs : 10625362017BFF4391187   path : /volume1/대성/5.등초본/씨에프-01/10625362/10625362_임혜자_초본.pdf
1건 중복파일 dict_kcs 정보 삭제


In [4]:
# dict 저장
print(len(dict_kcs))
write_kcs(path_kcs, dict_kcs)

33404


In [9]:
len(dict_kcs)

566532

### 중복파일인지 확인 : 채무자키 입히기 전

#### 단일 파일 - 채무자키 있음

In [ ]:
filePath = r"C:\Users\DATA\Desktop\압류추심요청서_2412\4.강제집행\20423304\20423304_장중돈_강제집행_2021타채116274.pdf"
k = os.path.split(filePath)[1][:8]
c = str(crc32_checksum(filePath))
s = str(os.path.getsize(filePath))
kcs = k + c + s
if kcs in dict_kcs :
    print(f"kcs : {kcs}   path : {dict_kcs[kcs]}")
else : 
    print("동일파일 없음")

#### 폴더 전체 - 채무자키 없음

In [ ]:
# key를 제거한 새로운 dict_cs 생성
dict_cs = {}
for key in dict_kcs:
    new_key = key[8:]  # 앞에서 8자리 문자를 지운 새로운 키 생성
    if new_key in dict_cs :
        dict_cs[new_key] = dict_cs[new_key] +","+ dict_kcs[key]
    else :
        dict_cs[new_key] = dict_kcs[key]

In [ ]:
path = r"D:\구 스캔파일\광주 공유폴더\■초본--작업보류\국민카드 초본 - 복사본"
file_list = read_filelist(path)
total = len(file_list)
path_exist = join(path, "exist")
cnt = 0 

for f in tqdm(file_list, total=total) :
    src = join(path, f)
    crc32 = str(crc32_checksum(src))
    size = str(os.path.getsize(src))
    cs = crc32 + size
    if cs in dict_cs :
        try : 
            os.rename(src, join(path_exist, f))
            cnt += 1
        except : 
            if not os.path.exists(path_exist) : os.mkdir(path_exist)
            os.rename(src, join(path_exist, f))
            cnt += 1
print(f"총 {total}개의 파일 중 {cnt}파일 중복")

### dict삭제 > 파일은 직접, 파일명 변경후 재업하면 수정작업과 동일

##### 단일파일 dict 삭제

In [ ]:
####################################
# 에브리띵에서 전체경로복사하면 편함, 채무자키만 유지하면 파일위치, 파일명는 상관없음
path_192 = r"\\192.168.0.75\솔림헬프\1.원인서류\DNP-02\20416486\20416486_김상호_원인서류.pdf"
####################################
삭제전 = len(dict_kcs)
delete_kcs(path_192, dict_kcs)
삭제후 = len(dict_kcs)
print(f'{삭제전 - 삭제후}건 중복파일 dict_kcs 정보 삭제')

##### 폴더 내 전체 파일 dict 삭제

In [3]:
path = r"C:\Users\DATA\Desktop\새 폴더" # r"\\192.168.0.75\스캔파일\대성 스캔파일 업로드\구스캔파일\새 폴더"
삭제전 = len(dict_kcs)

for f in os.listdir(path) :
    path_192 = os.path.join(path, f)
    if os.path.isfile(path_192) : 
        delete_kcs(path_192, dict_kcs)
        
삭제후 = len(dict_kcs)
print(f'{삭제전 - 삭제후}건 중복파일 dict_kcs 정보 삭제')


kcs : 204294012162C265510912   path : /volume1/솔림헬프/2.양도통지서/한울가람/20429401/20429401_보증인 최동근_양도통지서.pdf
kcs : 20429401961612BF521563   path : /volume1/솔림헬프/2.양도통지서/한울가람/20429401/20429401_보증인 최동근_양도통지서_(1).pdf
kcs : 204294018371A1A1566116   path : /volume1/솔림헬프/2.양도통지서/한울가람/20429401/20429401_오준태_양도통지서 1차.pdf
kcs : 204294015407ECCF2022477   path : /volume1/솔림헬프/1.원인서류/한울가람/20429401/20429401_오준태_원인서류.pdf
kcs : 20429403AA0B3DC1568458   path : /volume1/솔림헬프/2.양도통지서/한울가람/20429403/20429403_윤성한_양도통지서 1차.pdf
kcs : 204294032B3A8958313125   path : /volume1/솔림헬프/1.원인서류/한울가람/20429403/20429403_윤성한_원인서류.pdf
kcs : 20429661856BAB65574635   path : /volume1/솔림헬프/2.양도통지서/한울가람/20429661/20429661_㈜강산면옥_양도통지서 1차.pdf
kcs : 2042966168EEBC42722336   path : /volume1/솔림헬프/2.양도통지서/한울가람/20429661/20429661_㈜강산면옥_양도통지서.pdf
kcs : 204296610BA2E1251530087   path : /volume1/솔림헬프/1.원인서류/한울가람/20429661/20429661_㈜강산면옥_원인서류.pdf
kcs : 204296615EF07D61538355   path : /volume1/솔림헬프/2.양도통지서/한울가람/20429661/20429661_보증인 송국도_양도통지서.pdf
kcs

##### dict 파일로 저장

In [4]:
write_kcs(path_kcs, dict_kcs)
print(len(dict_kcs))

556357


##### 파일 삭제/이동

In [ ]:
# 관리제외 폴더로 이동
import shutil
src = path_192
dst_d = join(r"\\192.168.0.75\삭제예정파일\관리제외", src.split(os.path.sep)[-3], src.split(os.path.sep)[-2])
if not os.path.exists(dst_d) : os.makedirs(dst_d)
dst = join(dst_d, src.split(os.path.sep)[-1])
shutil.move(src, dst)

### dict 수정 
- 폴더 달라질 때는 그냥 새로 업로드하는게 낫겠다.

In [ ]:
########################################################### 20514967
src = r"\\192.168.0.75\솔림헬프\3.집행권원\DNP-01\20414371\20414371_조영민_집행권원.pdf"
dst = r"\\192.168.0.75\솔림헬프\3.집행권원\DNP-01\20414371\20414371 조영민 2023차전102996.pdf"
###########################################################

# 1) dict 수정
update_kcs(src, dst, dict_kcs)

In [ ]:
# 2) 파일수정 : dict보다 먼저 하면 kcs를 읽을 수가 없음
os.rename(src, dst)
print("파일명(경로) 수정 성공")

In [ ]:
# 3) dict 파일로 저장 # 약 20초 소요
write_kcs(path_kcs, dict_kcs)

#### 폴더 내 여러 파일 

In [ ]:
########################################################
path = r"\\192.168.0.75\솔림헬프\5.등초본\한울가람\20419713"
for f in os.listdir(path) :
    src = join(path, f)
    dst_file = re.sub("염직", "88염직", f)
    ####################################
    dst = join(path, dst_file)
    if src != dst :
        update_kcs(src, dst, dict_kcs)
        os.rename(src, dst)

In [ ]:
write_kcs(path_kcs, dict_kcs)